[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eafriyie28/beamgis/blob/main/docs/examples/time_slider.ipynb)

#### Uncomment the following line to install beamgis if needed.

In [ ]:
# !pip install beamgis

In [ ]:
import beamgis
import os

In [ ]:
# Set an environment variable with a specific key and value.
# This key is likely used for authentication or configuration purposes in the application.
os.environ["PLAK6d14428f432246128315488647ccf9b3"] = "12345"

In [ ]:
m = beamgis.Map(center=[38.2659, -103.2447], zoom=13)
m

In [ ]:
# Create a new map instance
m = beamgis.Map()

# Retrieve quarterly tiles from Planet using the provided API key
layers_dict = beamgis.planet_quarterly_tiles("PLAK6d14428f432246128315488647ccf9b3")

# Add a time slider to the map with the retrieved layers and a time interval of 1
m.add_time_slider(layers_dict, time_interval=1)

# Display the map
m

In [ ]:
m = beamgis.Map()
m.clear_layers()
layers_dict = beamgis.basemap_xyz_tiles()
m.add_time_slider(layers_dict, time_interval=1)
m

### Use the time slider to visualize COG assets found within STAC items.

In [ ]:
import ipyleaflet
import json
import requests

stac_api = "https://earth-search.aws.element84.com/v0"
search_endpoint = f"{stac_api}/search"

collection = "sentinel-s2-l2a-cogs"
payload = {
    "bbox": [
        -102.83340454101562,
        49.77860375256143,
        -102.41043090820312,
        50.05273014900257,
    ],
    "datetime": "2021-07-01T00:00:00Z/2021-10-01T12:31:12Z",
    "collections": [collection],
    "limit": 10,
    "query": {"eo:cloud_cover": {"gte": 0, "lte": 10}},
}

headers = {"Content-Type": "application/json"}

response = requests.request(
    "POST", search_endpoint, headers=headers, data=json.dumps(payload)
)

features = response.json()["features"]
features.sort(key=lambda x: x["properties"]["datetime"], reverse=False)

layers_dict = {}
for feature in features:
    feature_id = feature["id"]
    print(feature_id)

    url = beamgis.stac_tile(
        f"{stac_api}/collections/{collection}/items/{feature_id}", bands=["visual"]
    )
    tile_layer = ipyleaflet.TileLayer(
        url=url,
        name=feature_id,
    )
    layers_dict[feature_id] = tile_layer

m = beamgis.Map(center=[50.093079, -103.152825], zoom=11)
m.add_time_slider(layers_dict, time_interval=2)
m